In [6]:
import numpy

In [73]:
# -*- coding: utf-8 -*-
import vk
import time
from my_data import MyVKData
from tqdm import tqdm
from sql_connector import SQLConnector

class VkTableMaker():
    def __init__(self, title='Фитнес', table_name='Writers', params='Id INT PRIMARY KEY AUTO_INCREMENT, Name VARCHAR(25)', local = True):
        """
            title = what this table about
            table_name = what there is actual name of a table in database
            params = what table CREATE params do you want to add
            local = is it local database
        """
        self.sqlConnector = SQLConnector(local=local)
        self.title = title
        self.DATABASE = 'data2you1';
        self.table_name = '{0}.{1}'.format(self.DATABASE, table_name)
        self.params = params
        self.local = local
    def ask_yes(self, s):
        print s + '\ny/n?'
        ans = raw_input()
        return ans in ['y', 'Y', 'YES', 'yes']
    def config(self):
        if not self.local:
            cur = self._get_cursor()
            cur.execute("ALTER TABLE {0} MODIFY COLUMN {1} VARCHAR(255)  CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL".format(self.table_name, 'first_name'))
            cur.execute("ALTER TABLE {0} MODIFY COLUMN {1} VARCHAR(255)  CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL".format(self.table_name, 'last_name'))
            cur.execute("ALTER TABLE {0} MODIFY COLUMN {1} VARCHAR(255)  CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL".format(self.table_name, 'main_interest'))
    def rewrite_table_if_exist(self):
        cur = self._get_cursor()
        querry = "CREATE DATABASE IF NOT EXISTS {0}".format(self.DATABASE)
        cur.execute(querry)
        # unf-8 provide
        if self.ask_yes("Do you want to rewrite table " + self.table_name + " with title " + self.title):
            cur.execute("DROP TABLE IF EXISTS {0}".format(self.table_name))
            cur.execute("CREATE TABLE {0}({1})".format(self.table_name, self.params))
        self.config()
    def _get_cursor(self):
        sql_c = self.sqlConnector
        sql_c.connect()
        return sql_c.con.cursor()
    
    def add_rows(self, data):
        cur = self._get_cursor()
        for person in tqdm(data):
            try:
                querry = "INSERT INTO "+self.table_name + " (VKiD, first_name, last_name, main_interest) VALUES("+str(person.vk_id)+","+",".join(["'"+person.first_name+"'", "'"+person.last_name+"'","'"+person.interests[0]+"'"])+')'
                cur.execute(querry)
            except IndexError:
                print('empty string')
    def SELECT_ALL(self):
        cur = self._get_cursor()
        querry = "SELECT * FROM {0}".format(self.table_name)
        print(querry)
        cur.execute(querry)
        return cur.fetchall()

In [69]:
class VKPerson:
    vk_id = None
    first_name = None
    last_name = None
    interests = [None]
    def __init__(self, vk_id, first_name, last_name=None, interests=None):
        self.vk_id = vk_id
        self.first_name = unicode(first_name)
        self.last_name = unicode(last_name)
        self.interests = unicode(interests)
        
class Vkinterests:
    main_interest = 'Фитнес'
    popular_interests = dict()
    
    def __init__(self, main_interest='Фитнес'):
        self.main_interest = main_interest
        self.persons = []
        
    def get_data(self):
        
        session = vk.AuthSession(app_id=MyVKData.APP_ID, user_login=MyVKData.LOGIN, user_password=MyVKData.PASSWORD())
        vkapi = vk.API(session)
        offs = 0
        all_interests = dict()
        for i in tqdm(range(10)):
            try:
                usrs = vkapi.users.search(interests=self.main_interest, extended=1, fields='interests', offset=offs)
                count = usrs[0]
                usrs = usrs[1:]
                offs += len(usrs) + 1
                for usr in usrs:
                    person = VKPerson(usr['uid'], usr['first_name'])
                    if ('last_name' in usr):
                        person.last_name = usr['last_name']
                    if ('interests' in usr):
                        person.interests = usr['interests'].split(',')
                        for inter in usr['interests'].split(','):
                            inter = inter.strip()
                            if inter in all_interests:
                                all_interests[inter] += 1
                            else:
                                all_interests[inter] = 1
                    self.persons.append(person)
                time.sleep(0.3)
            except ReadTimeout:
                time.sleep(1)
                print("ReadTimeOut")
        self.popular_interests = all_interests
        print("{0} users successfully downloaded".format(len(self.persons)))
        
    def print_all_interests(self):
        for interest in sorted(self.popular_interests.items(), key=lambda x: x[1], reverse=True):
            print interest[0], interest[1]
    
    def add_all_data_to_sql(self, local, append):
        table = VkTableMaker(
            title='Таблица вк по интересам',
            table_name = 'VK_interests',
            params = 'Id INT PRIMARY KEY AUTO_INCREMENT, VKiD INT, first_name VARCHAR(25), last_name VARCHAR(25), main_interest VARCHAR(25)',
            local = local
        )
        if (not append):
            table.rewrite_table_if_exist()
        if table.ask_yes("Do you want to add {0} row in table {1}? ".format(len(self.persons), table.table_name)):
            table.add_rows(self.persons)
        else:
            return
        self.table = table

        







In [70]:
vkint = Vkinterests()
vkint.get_data()

100%|██████████| 10/10 [00:04<00:00,  2.12it/s]

195 users successfully downloaded


In [71]:
for i in vkint.persons:
    print i.first_name

Яна
Денис
Елена
Илона
Юрий
Анастасия
Катя
Денис
Светлана
Павел
Павел
Ярик
Михаил
Кристина
Соня
Яна
Оксана
Роман
Татьяна
Татьяна
Оксана
Кристина
Милена
Павел
Владислав
Агнета
Марина
Alua
Pavel
Антон
Ренат
Sarkis
Костя
Анастасия
Катрина
Анастасия
Диманыч
Владимир
Владимир
Вероника
Дмитрий
Мария
Денис
Василий
Яна
Таня
Александр
Светлана
Эдуард
Людмила
Юрий
Ольга
Денис
Илья
Ирина
Лина
Александр
Анастасия
Кристина
Archie
Оленька
Яно4kа
Юлия
Кристина
Олесь
Андрей
Тимур
Роман
Алиса
Игорь
Ксения
Тарас
Ольга
Дарья
Арина
Катюша
Александр
Вячеслав
Наталья
Irinka
Анжелика
Сара
Андрей
Кристина
Lola
Виктория
Владислав
Инга
Александр
Вячеслав
Рубин
Оленька
Delf
Елена
Леся
Даша
Элеонора
Юра
Екатерина
Александра
Дмитрий
Александр
Наталья
Павел
Мария
Юленька
Сергей
Богдана
Андрей
Анна
Викусинька
Евгений
Наташа
Дмитрий
Маргарита
Рита
Саня
Vitaliy
Yana
Анна
Нона
Ирина
Регина
Armen
Алёна
Максим
Алексей
Анжелика
Софи
Альберт
Андрей
Юля
Валерия
Верушка
Алексей
Максим
Андрей
Максим
Елена
Натали
Анастасия
Юля


In [74]:
vkint.add_all_data_to_sql(local=False, append=False)

/Users/istar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:35: Warning: Can't create database 'data2you1'; database exists


Do you want to rewrite table data2you1.VK_interests with title Таблица вк по интересам
y/n?
y
Do you want to add 195 row in table data2you1.VK_interests? 
y/n?
y


100%|██████████| 195/195 [01:04<00:00,  2.99it/s]


In [75]:
vkint.table.SELECT_ALL()

SELECT * FROM data2you1.VK_interests


((1L, 9562528L, '\xd0\xaf\xd0\xbd\xd0\xb0', '\xd0\x91\xd1\x83\xd1\x86', 'N'),
 (2L,
  210841413L,
  '\xd0\x94\xd0\xb5\xd0\xbd\xd0\xb8\xd1\x81',
  '\xd0\x90\xd0\xbd\xd0\xb4\xd1\x80\xd1\x96\xd1\x86\xd0\xb0',
  'N'),
 (3L,
  33850323L,
  '\xd0\x95\xd0\xbb\xd0\xb5\xd0\xbd\xd0\xb0',
  '\xd0\xa2\xd1\x80\xd0\xb5\xd1\x82\xd1\x8c\xd1\x8f\xd0\xba\xd0\xbe\xd0\xb2\xd0\xb0',
  '\xd0\x9c\xd1\x83\xd0\xb7\xd1\x8b\xd0\xba\xd0\xb0. \xd0\xa4\xd0\xb8\xd1\x82\xd0\xbd\xd0\xb5\xd1\x81. \xd0\x99\xd0\xbe\xd0\xb3\xd0\xb0. \xd0\x9f\xd1\x81\xd0\xb8\xd1\x85\xd0\xbe\xd0\xbb\xd0\xbe\xd0\xb3\xd0\xb8\xd1\x8f.'),
 (4L,
  151684580L,
  '\xd0\x98\xd0\xbb\xd0\xbe\xd0\xbd\xd0\xb0',
  '\xd0\x9d\xd0\xbe\xd0\xb2\xd0\xbe\xd1\x81\xd1\x91\xd0\xbb\xd0\xbe\xd0\xb2\xd0\xb0',
  'N'),
 (5L,
  6764187L,
  '\xd0\xae\xd1\x80\xd0\xb8\xd0\xb9',
  '\xd0\xa1\xd0\xbf\xd0\xb0\xd1\x81\xd0\xbe\xd0\xba\xd1\x83\xd0\xba\xd0\xbe\xd1\x86\xd0\xba\xd0\xb8\xd0\xb9',
  '\xd0\x91\xd0\xbe\xd0\xb4\xd0\xb8\xd0\xb1\xd0\xb8\xd0\xbb\xd0\xb4\xd0\xb8\xd0\xbd\xd0

In [28]:
sql_c = SQLConnector()
sql_c.connect()
cur = sql_c.con.cursor()
cur.execute("SELECT * FROM data2you1.VK_interests")
print(cur.fetchall())
cur.close()

((1L, 9562528L, u'???', u'???', None), (2L, 210841413L, u'?????', u'???????', None), (3L, 33850323L, u'?????', u'??????????', u'??????. ??????. ????. ???'), (4L, 33850323L, u'?????', u'??????????', u'??????. ??????. ????. ???'), (5L, 151684580L, u'?????', u'??????????', None), (6L, 6764187L, u'????', u'?????????????', u'???????????'), (7L, 6764187L, u'????', u'?????????????', u'???????????'), (8L, 209452406L, u'?????????', u'????????', None), (9L, 264356807L, u'????', u'????', u''), (10L, 264356807L, u'????', u'????', u''), (11L, 1784535L, u'?????', u'??????', None), (12L, 231817577L, u'????????', u'??????', None), (13L, 40946004L, u'?????', u'???????', u'???????? ???????'), (14L, 40946004L, u'?????', u'???????', u'???????? ???????'), (15L, 72991632L, u'?????', u'??????', None), (16L, 3881134L, u'????', u'??????', u'?????????? ???????'), (17L, 3881134L, u'????', u'??????', u'?????????? ???????'), (18L, 175L, u'??????', u'??????????', None), (19L, 211091295L, u'????????', u'?????????', 

ÐÐ»ÐµÐ½Ð°
